In [ ]:
# тест-суит - Owasp-Benchmark Java


import requests
import csv
import pandas as pd
from google.colab import drive
import ast
import re

# Подключаем Google Диск
drive.mount('/content/drive')


# Загрузка данных из CSV файлов
formatted_issues_path = '/content/drive/My Drive/semgerep_02_04_2025.csv'
real_vulnerabilities_path = '/content/drive/My Drive/expectedresults-1.2.csv'  # Укажите путь к вашему файлу с реальными уязвимостями

# Чтение реальных уязвимостей
with open(real_vulnerabilities_path, mode='r', encoding='utf-8') as real_vul_file:
    real_vulnerabilities = list(csv.reader(real_vul_file, delimiter=';'))
    # Убираем заголовки
    real_vulnerabilities = real_vulnerabilities[1:]  # Пропускаем первую строку

# Чтение форматированных проблем
with open(formatted_issues_path, mode='r', encoding='utf-8') as formatted_issues_file:
    formatted_iss = list(csv.reader(formatted_issues_file, delimiter=';'))
    # Убираем заголовки
    formatted_iss = formatted_iss[1:]  # Пропускаем первую строку

formatted_issues = formatted_iss

# Вывод названий колонок и размера
print("Размер real_vulnerabilities:", len(real_vulnerabilities))


real_vulnerabilities_true = []  # Создаем пустой список для хранения соответствующих элементов

for real_issue in real_vulnerabilities:
    if 'true' in str(real_issue):  # Проверяем, содержится ли строка 'true' в элементе
      real_vulnerabilities_true.append(real_issue)

print(len(real_vulnerabilities_true))

# Инициализация счетчиков
TP = 0  # True Positives
FP = 0  # False Positives
FN = 0  # False Negatives

result = {}
negative = {}
for real_issue in real_vulnerabilities_true:
  cleaned_string = str(real_issue).strip("[]'")
  data_list = cleaned_string.split(',')
  benchmark_name = data_list[0]
  result[benchmark_name] = []
  negative[benchmark_name] = []

# Сравнение данных
for real_issue in real_vulnerabilities_true:
    # Удаление квадратных скобок и лишних пробелов
    cleaned_string = str(real_issue).strip("[]'")
    # Разделение строки на элементы списка
    data_list = cleaned_string.split(',')
    benchmark_name = data_list[0]
    cwe = data_list[3]
    # Проверяем, есть ли соответствующая запись в formatted_issues
    matching_issues = []
    not_mat_issues = []
    for formatted_issue in formatted_issues:
      cleaned_string = str(formatted_issue)[1:-1]
      form_list = cleaned_string.split(',')
      file_name = str(re.findall(r'BenchmarkTest\d+', form_list[0])[0])
      if file_name==benchmark_name and int(cwe) == int(form_list[1].replace("'", "")):
        TP += 1


# Дополнительный блок для нахождения ложных отрицаний
for real_issue in real_vulnerabilities_true:
    cleaned_string = str(real_issue).strip("[]'")
    # Разделение строки на элементы списка
    data_list = cleaned_string.split(',')
    benchmark_name = data_list[0]
    cwe = data_list[3]


    # Проверяем, есть ли это реальное нарушение среди отформатированных
    if not any(
        (str(re.findall(r'BenchmarkTest\d+', str(formatted_issue)[1:-1].split(",")[0])[0])==benchmark_name and cwe in str(formatted_issue)[1:-1].split(",")[1])
        for formatted_issue in formatted_issues
    ):
        # Если не найдено в formatted_issues, добавляем к ложным отрицаниям
      FN += 1

for formatted_issue in formatted_issues:
    cleaned_string = str(formatted_issue)[1:-1]
    form_list = cleaned_string.split(',')
    file_name = str(re.findall(r'BenchmarkTest\d+', form_list[0])[0])

    if not any(
        (file_name in str(real_issue) and int(str(real_issue).strip("[]'").split(",")[3])==int(form_list[1].replace("'", "")))
        for real_issue in real_vulnerabilities_true
    ):
        # Если не найдено в formatted_issues, добавляем к ложным отрицаниям
      FP += 1


# Вывод результатов
# Расчет метрик
recall = TP / (TP + FN)
precision = TP / (TP + FP)
f1_score = 2*(recall*precision)/(recall+precision)
# Вывод результатов
print(f"Recall: {recall:.2f}")
print(f"precision: {precision:.2f}")
print(f"F1-score: {f1_score:.2f}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Размер real_vulnerabilities: 2740
1415
Recall: 0.21
precision: 0.20
F1-score: 0.21


In [ ]:
# тест-суит - OWASP Security Shepherd

import requests
import csv
import pandas as pd
from google.colab import drive
import ast
import re

# Подключаем Google Диск
drive.mount('/content/drive')


# Загрузка данных из CSV файлов
formatted_issues_path = '/content/drive/My Drive/semgrep_shepherd.csv'
real_vulnerabilities_path = '/content/drive/My Drive/output.csv'

# Чтение реальных уязвимостей
with open(real_vulnerabilities_path, mode='r', encoding='utf-8') as real_vul_file:
    real_vulnerabilities = list(csv.reader(real_vul_file, delimiter=';'))
    # Убираем заголовки
    real_vulnerabilities = real_vulnerabilities[1:]  # Пропускаем первую строку

# Чтение форматированных проблем
with open(formatted_issues_path, mode='r', encoding='utf-8') as formatted_issues_file:
    formatted_iss = list(csv.reader(formatted_issues_file, delimiter=';'))
    # Убираем заголовки
    formatted_iss = formatted_iss[1:]  # Пропускаем первую строку

formatted_issues = formatted_iss


# Вывод названий колонок и размера
print("Размер real_vulnerabilities:", len(real_vulnerabilities))

print("Размер real_vulnerabilities:", len(formatted_issues))


# Инициализация счетчиков
TP = 0  # True Positives
FP = 0  # False Positives
FN = 0  # False Negatives

def getData_formatted_issue(formatted_issue, real_issue):
  toReturn = []
  pattern = r'/([^/]+\.java)#L(\d+)'
  match = re.search(pattern, formatted_issue[0].split(',')[0])
  file_name = ""
  line_number = -1
  if match:
    file_name = match.group(1)  # Название файла
    line_number = match.group(2)  # Номер строки
  cleaned_string = str(formatted_issue)[1:-1]
  form_list = cleaned_string.split(',')
  cweList = str(formatted_issue)[1:-1].replace(form_list[0]+",","").replace('[',"").replace("'","").replace("]","").replace("'","").split(",")
  formatted_cwe = []
  if(cweList!=['']):
    formatted_cwe = [f'cwe:{item}' for item in cweList]
  matches = []
  if formatted_cwe!=[]:
    matches = [item for item in formatted_cwe if item in str(real_issue)]
  toReturn.append(file_name)
  toReturn.append(line_number)
  toReturn.append(matches)
  return toReturn

# Сравнение данных
for real_issue in real_vulnerabilities:
    # Удаление квадратных скобок и лишних пробелов
    cleaned_string = str(real_issue).strip("[]'")
    # Разделение строки на элементы списка
    data_list = cleaned_string.split(',')
    path = data_list[1];
    pattern = r"([^/]+\.java)$"

    #  Используем метод search
    matcher = re.search(pattern, path)
    benchmark_name = ""
    if matcher:
      benchmark_name = matcher.group(1)
    rule = data_list[0]
    line = data_list[2]
    for formatted_issue in formatted_issues:
      pattern = r'/([^/]+\.java)#L(\d+)'
      match = re.search(pattern, formatted_issue[0].split(',')[0])
      file_name = ""
      line_number = -1
      if match:
        file_name = match.group(1)  # Название файла
        line_number = match.group(2)  # Номер строки
      cleaned_string_form = str(formatted_issue)[1:-1]
      form_list = cleaned_string_form.split(',')
      cweList = str(formatted_issue)[1:-1].replace(form_list[0]+",","").replace('[',"").replace("'","").replace("]","").replace("'","").split(",")
      formatted_cwe = []
      if(cweList!=['']):
        formatted_cwe = [f'cwe:{item}' for item in cweList]
      matches = []
      if len(formatted_cwe)>0:
        matches = [item for item in formatted_cwe if item in str(real_issue)]
      if file_name==benchmark_name and len(matches)!=0 and int(line_number) == int(line):
      #and int(line)==int(form_list[2]) :
        TP += 1
print(TP)

def getData_real_issue(real_issue):
  toReturn = []
  # Удаление квадратных скобок и лишних пробелов
  cleaned_string = str(real_issue).strip("[]'")
    # Разделение строки на элементы списка
  data_list = cleaned_string.split(',')
  path = data_list[1];
  pattern = r"([^/]+\.java)$"

    #  Используем метод search
  matcher = re.search(pattern, path)
  benchmark_name = ""
  if matcher:
    benchmark_name = matcher.group(1)
    rule = data_list[0]
    line = data_list[2]
  toReturn.append(file_name)
  toReturn.append(line_number)
  return toReturn


# Дополнительный блок для нахождения ложных отрицаний
for real_issue in real_vulnerabilities:
    cleaned_string = str(real_issue).strip("[]'")
    data_list = cleaned_string.split(',')
    path = data_list[1]
    pattern = r"([^/]+\.java)$"

    matcher = re.search(pattern, path)
    benchmark_name = ""
    if matcher:
        benchmark_name = matcher.group(1)

    rule = data_list[0]
    line = data_list[2]

    # Проверяем, есть ли это реальное нарушение среди отформатированных
    if not any(
        (int(getData_formatted_issue(formatted_issue, str(real_issue))[1])==int(line) and len(getData_formatted_issue(formatted_issue, str(real_issue))[2])>0 and getData_formatted_issue(formatted_issue, str(real_issue))[0] == benchmark_name)
        for formatted_issue in formatted_issues
    ):
        # Если не найдено в formatted_issues, добавляем к ложным отрицаниям
        FN += 1

for formatted_issue in formatted_issues:
    if not any(
        (int(getData_formatted_issue(formatted_issue, real_issue)[1])==int(getData_real_issue(real_issue)[1]) and len(getData_formatted_issue(formatted_issue, real_issue)[2])>0 and getData_formatted_issue(formatted_issue, real_issue)[0] == getData_real_issue(real_issue)[0])
        for real_issue in real_vulnerabilities
    ):
        FP += 1

# Расчет метрик
recall = TP / (TP + FN)
precision = TP / (TP + FP)
f1_score = 2*(recall*precision)/(recall+precision)
# Вывод результатов
print(f"Recall: {recall:.2f}")
print(f"precision: {precision:.2f}")
print(f"F1-score: {f1_score:.2f}")



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Размер real_vulnerabilities: 200
Размер real_vulnerabilities: 376
17
Recall: 0.08
precision: 0.04
F1-score: 0.06
